$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Banqi AI

Jared Zymbaluk

In [3]:
import numpy as np
from random import shuffle
import time
import math
import copy

In [4]:
class piece(object):
    name = ""
    power = ""
    color = ""
    flipped = False
    def __init__(self, n, p, c):
        self.name = n
        self.power = p
        self.color = c

In [5]:
class Move(object):
    mType = ""
    move = ""
    power = ""
    def __init__(self, t, m, p):
        self.mType = t
        self.move = m
        self.power = p
    

In [6]:
def getPower(m):
    return m.power

In [7]:
def createGame():
    game = []
    
    #add correct number of pieces to game board
    ws = piece("soldier",1,"w")
    bs = piece("soldier",1,"b")
    for i in range(5):
        game.append(copy.copy(ws))
        game.append(copy.copy(bs))
        
    wh = piece("horse", 2, "w")
    bh = piece("horse", 2, "b")
    for i in range(2):
        game.append(copy.copy(wh))
        game.append(copy.copy(bh))
    
    wc = piece("chariot", 3, "w")
    bc = piece("chariot", 3, "b")
    for i in range(2):
        game.append(copy.copy(wc))
        game.append(copy.copy(bc))
    
    we = piece("elephant", 4, "w")
    be = piece("elephant", 4, "b")
    for i in range(2):
        game.append(copy.copy(we))
        game.append(copy.copy(be))
        
    wa = piece("advisor", 5, "w")
    ba = piece("advisor", 5, "b")
    for i in range(2):
        game.append(copy.copy(wa))
        game.append(copy.copy(ba))
        
    wg = piece("general", 6, "w")
    bg = piece("general", 6, "b")
    for i in range(1):
        game.append(copy.copy(wg))
        game.append(copy.copy(bg))
        
    wc = piece("cannon", 0, "w")
    bc = piece("cannon", 0, "b")
    for i in range(2):
        game.append(copy.copy(wc))
        game.append(copy.copy(bc))
    
    
    shuffle (game)
    shuffle (game)
    shuffle (game)
    return game

In [8]:
def printGame(game):
    i = 1
    print("1 | ", end = '')
    for p in game:
        
        if isinstance(p, piece):
            if p.flipped == False:
                print("?  | ", end = '')
            else:
                print(p.color + str(p.power) + " | ", end = '')
            
        else:
            print("   | ", end = '')
        
        if i > 7 and i % 8 == 0 and i!= 32:
                print("\n")
                print(str((i // 8) + 1) + " | ", end = '')
        
        i = i + 1
        
    print()   
    print("    A    B    C    D    E    F    G    H")
        

In [9]:
def getMoves(game, color):
    i = 0;
    moves = []
    for p in game:
        if isinstance(p, piece):
            if p.flipped == False:
                moves.append(Move("flip",("flip",i),-10))
            else:
                if p.color == color:
                    check = []
                    if i>0 and i%8!=0: check.append(i-1) #check left
                    if i<31 and i%7!=0: check.append(i+1) #check right
                    if i-8>=0: check.append(i-8) #check up
                    if i+8<=31: check.append(i+8) #check down
                        
                    for c in check:
                        if isinstance(game[c],piece):
                            if game[c].flipped and game[c].color != color and (game[c].power <= p.power or p.power == 0 or (p.power == 1 and game[c].power==6)): #power is less, or is cannon
                                m = Move("capture",(i,c),game[c].power-p.power)
                                moves.append(m)
                        else:
                            moves.append(Move("move",(i,c),-11))             
        i=i+1
    return moves       

In [10]:
def makeMove(game, m):
    mFrom, mTo = m.move
    
    gCopy = copy.deepcopy(game)
    if m.mType == "flip":
        gCopy[mTo].flipped = True
        
    else:
        gCopy[mTo] = gCopy[mFrom]
        gCopy[mFrom] = " "
    return gCopy
        
    

In [11]:
def isOver(game,color):
    for p in game:
        if isinstance(p,piece):
            if p.color !=color:
                return False
    return True

In [12]:
def score(game, color):
    power = 0
    oPower = 0
    for p in game:
        if isinstance(p,piece):
            if p.color == color and p.flipped:
                if p.power == 0:
                    power = power + 7
                else:
                    power = power + p.power
            if p.color != color and p.flipped:
                if p.power == 0:
                    oPower = oPower + 7
                else:
                    oPower = oPower + p.power
    i=0
    for g in game:
        if isinstance(game[i],piece):
            if game[i].color == color:
                power = power - inDanger(game, color, i)
            if game[i].color != color:
                oPower = oPower - inDanger(game, color, i)
        i=i+1
                
    return power-oPower

In [13]:
def inDanger(game, color, i):
    check = []
    if i>0 and i%8!=0: check.append(i-1) #check left
    if i<31 and i%7!=0: check.append(i+1) #check right
    if i-8>=0: check.append(i-8) #check up
    if i+8<=31: check.append(i+8) #check down
    penalty = 0                    
    for c in check:
        if isinstance(game[c],piece):
            if game[c].flipped and game[c].color != color and (game[c].power >= game[i].power or game[c].power == 0 or (game[c].power == 1 and game[i].power==6)): #power is less, or is cannon
                   penalty = penalty + 1
    return penalty

In [14]:
def interpretSquare(square):
    letter,number = list(square)
    res = ord(letter)-ord('A')
    res = res + ((int(number)-1)*8)
    return res

In [15]:
def interpretMove(num):
    if num == "flip":
        return num
    else:
        res = ""
        digit = (num//8)+1
        return str(digit)
        
        

In [16]:
def negamaxIDSab(game, depthLimit, color):
    for depth in range(depthLimit):
        bestValue,bestMove = negamaxab(game,depth,float("-inf"),float("inf"), color)
        if bestValue == "done":
            return bestValue,bestMove
    return bestValue,bestMove

In [20]:
def negamaxab(game, depthLeft, alpha, beta, color):
    opponentColor = ""
    if color == "w":opponentColor = "b"
    if color == "b":opponentColor = "w"
    # If at terminal state or depth limit, return utility value and move None
    if isOver(game, color) or depthLeft == 0:
        return score(game,color), None
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    moves = getMoves(game, color)
    shuffle(moves)
    moves.sort(key=lambda x: x.power, reverse=True) #sorts list so HIGHEST score moves are first
    if moves[0].power > -2:
        return moves[0].power,moves[0]
    for move in moves:
        if move.mType == "flip":
            value = 2
        
        else:
            # Apply a move to current state
            newBoard = makeMove(game,move)
            # Use depth-first search to find eventual utility value and back it up.
            #  Negate it because it will come back in context of next player
            value, _ = negamaxab(newBoard, depthLeft-1, -beta, -alpha, opponentColor)   
            # Remove the move from current state, to prepare for trying a different move
            #game.unmakeMove(move)
            if value is None:
                continue
            value = - value
            #return early if best value is greater than beta
            if value >= beta:
                return value, bestMove
            alpha = max(alpha,value)
        #update best value
        if bestValue is None or value > bestValue:
                # Value for this move is better than moves tried so far from this state.
                bestValue, bestMove = value, move
        
    return bestValue, bestMove

In [21]:
def playGame(game,color,depthLimit):
    while not isOver(game,color):
        score,move = negamaxIDSab(game,depthLimit, color)
        if move == None :
            print('move is None. Stopping.')
            break
        game = makeMove(game,move)
        aifrom,aito = move.move
        print("Your opponent made the move " + interpretMove(aifrom) + " " + interpretMove(aito))
        printGame(game)
        print("\n")
        if not isOver(game, color):
            mfrom =""
            mto = ""
            mfrom = input("What's your move?: ")
            while mfrom != "flip" and len(mfrom)>2:
                mfrom = input("Incorrect input What's your move?: ")
            print()
            if mfrom == "flip":
                mto = input("Which piece?: ")
                while len(mto) > 2:
                    mto = input("Incorrect input which piece?: ")
                m = Move("flip",(mfrom,interpretSquare(mto)),-1)
            else:
                mto = input("To?: ")
                while len(mto) > 2:
                    mto = input("Incorrect input, to?: ")
                m = Move("user",(interpretSquare(mfrom),interpretSquare(mto)),-1)
            game = makeMove(game,m)

In [22]:
game = createGame()
playGame(game,"w",2)

Your opponent made the move flip 4
1 | ?  | ?  | ?  | ?  | ?  | ?  | ?  | ?  | 

2 | ?  | ?  | ?  | ?  | ?  | ?  | ?  | ?  | 

3 | ?  | ?  | ?  | ?  | ?  | ?  | ?  | ?  | 

4 | ?  | ?  | ?  | ?  | ?  | b0 | ?  | ?  | 
    A    B    C    D    E    F    G    H


What's your move?: flip

Which piece?: H4
Your opponent made the move flip 1
1 | ?  | ?  | ?  | ?  | ?  | ?  | ?  | b5 | 

2 | ?  | ?  | ?  | ?  | ?  | ?  | ?  | ?  | 

3 | ?  | ?  | ?  | ?  | ?  | ?  | ?  | ?  | 

4 | ?  | ?  | ?  | ?  | ?  | b0 | ?  | b5 | 
    A    B    C    D    E    F    G    H


What's your move?: flip

Which piece?: B2
Your opponent made the move flip 3
1 | ?  | ?  | ?  | ?  | ?  | ?  | ?  | b5 | 

2 | ?  | w1 | ?  | ?  | ?  | ?  | ?  | ?  | 

3 | ?  | ?  | ?  | ?  | ?  | ?  | w0 | ?  | 

4 | ?  | ?  | ?  | ?  | ?  | b0 | ?  | b5 | 
    A    B    C    D    E    F    G    H


What's your move?: flip

Which piece?: G4
Your opponent made the move 3 4
1 | ?  | ?  | ?  | ?  | ?  | ?  | ?  | b5 | 

2 | ?  | w1 |

KeyboardInterrupt: 